<a href="https://colab.research.google.com/github/LeeYoungWook/dd/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gan**

![대체 텍스트](https://raw.githubusercontent.com/LeeYoungWook/dd/master/sadsadsaa.PNG)




In [0]:

import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np

In [0]:

# 하이퍼파라미터
EPOCHS = 500
BATCH_SIZE = 100
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("Using Device:", DEVICE)

Using Device: cuda


In [0]:

# Fashion MNIST 데이터셋
trainset = datasets.FashionMNIST(
    './.data',
    train=True,
    download=True,
    transform=transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.5,), (0.5,))
    ])
)
train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True
)


Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw
Processing...
Done!


In [0]:

# 생성자 (Generator)
G = nn.Sequential(
        nn.Linear(64, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 784),
        nn.Tanh())

In [0]:
# 판별자 (Discriminator)
D = nn.Sequential(
        nn.Linear(784, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 1),
        nn.Sigmoid())

In [0]:
# 모델의 가중치를 지정한 장치로 보내기
D = D.to(DEVICE)
G = G.to(DEVICE)

# 이진 크로스 엔트로피 (Binary cross entropy) 오차 함수와
# 생성자와 판별자를 최적화할 Adam 모듈
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)

In [0]:
#결과값: 1시간 30분 걸림

total_step = len(train_loader)
for epoch in range(EPOCHS):
    for i, (images, _) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(DEVICE)
        
        # '진짜'와 '가짜' 레이블 생성
        real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)
        
        # 판별자가 진짜 이미지를 진짜로 인식하는 오차를 예산
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # 무작위 텐서로 가짜 이미지 생성
        z = torch.randn(BATCH_SIZE, 64).to(DEVICE)
        fake_images = G(z)
        
        # 판별자가 가짜 이미지를 가짜로 인식하는 오차를 계산
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # 진짜와 가짜 이미지를 갖고 낸 오차를 더해서 판별자의 오차 계산
        d_loss = d_loss_real + d_loss_fake

        # 역전파 알고리즘으로 판별자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 생성자가 판별자를 속였는지에 대한 오차를 계산
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        
        # 역전파 알고리즘으로 생성자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    # 학습 진행 알아보기
    print('Epoch [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
          .format(epoch, EPOCHS, d_loss.item(), g_loss.item(), 
                  real_score.mean().item(), fake_score.mean().item()))

Epoch [0/500], d_loss: 0.0944, g_loss: 3.8645, D(x): 0.97, D(G(z)): 0.05
Epoch [1/500], d_loss: 0.0503, g_loss: 5.7646, D(x): 0.99, D(G(z)): 0.02
Epoch [2/500], d_loss: 0.0158, g_loss: 6.0480, D(x): 1.00, D(G(z)): 0.01
Epoch [3/500], d_loss: 0.0356, g_loss: 5.5535, D(x): 1.00, D(G(z)): 0.03
Epoch [4/500], d_loss: 0.0207, g_loss: 6.9220, D(x): 0.99, D(G(z)): 0.01
Epoch [5/500], d_loss: 0.0802, g_loss: 6.8762, D(x): 0.98, D(G(z)): 0.02
Epoch [6/500], d_loss: 0.1711, g_loss: 6.0730, D(x): 0.96, D(G(z)): 0.06
Epoch [7/500], d_loss: 0.1716, g_loss: 4.7396, D(x): 0.97, D(G(z)): 0.09
Epoch [8/500], d_loss: 0.1305, g_loss: 3.9854, D(x): 0.97, D(G(z)): 0.07
Epoch [9/500], d_loss: 0.2602, g_loss: 5.1508, D(x): 0.93, D(G(z)): 0.04
Epoch [10/500], d_loss: 0.3055, g_loss: 4.3838, D(x): 0.88, D(G(z)): 0.03
Epoch [11/500], d_loss: 0.1315, g_loss: 4.7074, D(x): 0.98, D(G(z)): 0.09
Epoch [12/500], d_loss: 0.2522, g_loss: 5.9269, D(x): 0.91, D(G(z)): 0.04
Epoch [13/500], d_loss: 0.3634, g_loss: 3.5211, 

# **기존 GAN의 한계와 이를 해결하기 위한 다양한 GAN의 등장**

1. GAN은 결과가 불안정하다
기존 GAN만 가지고는 좋은 성능이 잘 안나왔다.

2. Black-box method
Neural Network 자체의 한계라고 볼 수 있는데, 결정 변수나 주요 변수를 알 수 있는 다수의 머신러닝 기법들과 달리 Neural Network은 처음부터 끝까지 어떤 형태로 그러한 결과가 나오게 되었는지 그 과정을 알 수 없다.

3. Generative Model 평가
GAN은 결과물 자체가 새롭게 만들어진 Sample 이다. 이를 기존 sample과 비교하여 얼마나 비슷한 지 확인할 수 있는 정량적 척도가 없고, 사람이 판단하더라도 이는 주관적 기준이기 때문에 얼마나 정확한지, 혹은 뛰어난지 판단하기 힘들다.

# **CGAN**

![CGAN](https://github.com/LeeYoungWook/dd/blob/master/CGAN1.PNG?raw=true)


# **DCGAN**

![DCGAN1](https://github.com/LeeYoungWook/dd/blob/master/DCAN1.PNG?raw=true)

![대체 텍스트](https://github.com/LeeYoungWook/dd/blob/master/DCGAN2.PNG?raw=true)


# **SGAN**


![대체 텍스트](https://github.com/LeeYoungWook/dd/blob/master/SGAN1.PNG?raw=true)

![대체 텍스트](https://github.com/LeeYoungWook/dd/blob/master/SGAN2.PNG?raw=true)

# **Defect gan**

![Defect gan](https://github.com/LeeYoungWook/dd/blob/master/Defect%20gan.PNG?raw=true)

**특징 및 활용**

![특징](https://github.com/LeeYoungWook/dd/blob/master/%ED%8A%B9%EC%A7%95.PNG?raw=true)